In [5]:
import numpy as np
from glob import glob
from sklearn.model_selection import train_test_split
from tensorflow.keras.layers import Input,Dense,Flatten
from tensorflow.keras.models import Model,Sequential
from tensorflow.keras.applications.inception_v3 import InceptionV3,preprocess_input
from tensorflow.keras.preprocessing import image

In [ ]:
train_path = (r"BIRDS/train")
test_path =  (r"BIRDS/test")

In [ ]:
img_size = [256,256]

In [ ]:
incpt = InceptionV3 (input_shape= img_size+[3], weights='imagenet', include_top= False)

In [ ]:
for layers in incpt.layers:
    layers.trainable = False

In [ ]:
folder = glob("BIRDS/train*")

In [ ]:
x = Flatten()(incpt.output)

In [ ]:
prediction = Dense(len(folder),activation = 'softmax')(x)

In [ ]:
model = Model (incpt.input,outputs= prediction)
model.summary()

In [ ]:
model.compile(loss='categorical_crossentropy',optimizer ='adam', metrics = ['accuracy'])

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [ ]:
train_datagen = ImageDataGenerator(rescale = 1./255, shear_range = 0.2,zoom_range = 0.2, horizontal_flip=True)
test_datagen = ImageDataGenerator(rescale=1./255)

In [ ]:
train_set = train_datagen.flow_from_directory(train_path,target_size=(256,256),batch_size=32, class_mode = 'categorical')
test_set = test_datagen.flow_from_directory(test_path,target_size=(256,256),batch_size=32, class_mode = 'categorical')

In [ ]:
classes = train_set.class_indices
classes

In [ ]:
import json
with open ("classes.json",'w') as f:
    json.dump(classes,f)

In [ ]:
fit = model.fit(train_set, validation_data= test_set, epochs=10, steps_per_epoch=len(train_set),
                validation_steps=len(test_set))

In [ ]:
model.save("model.h5")